In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [ ]:
FeatureSetB = pd.read_csv('MIMIC_Windows/FSB_5Ws.csv')
FeatureMortality = pd.read_csv('MIMIC_Windows/FSB_Mortality.csv')

In [ ]:
numberOfWindows = 5
numberOfFeatures = 20

In [ ]:
Subject_id = pd.DataFrame()
Subject_id = FeatureSetB.subject_id
Subject_id.drop_duplicates(keep = 'first', inplace = True)
Subject_id.reset_index(drop=True,inplace=True)
NumSubjects = len(Subject_id)

In [ ]:
X = FeatureMortality['subject_id'].equals(Subject_id)

In [ ]:
FeatureSet = FeatureSetB.drop(['subject_id'], axis=1)

In [ ]:
FeatureSet = FeatureSet.to_numpy()

In [ ]:
Labels = pd.DataFrame()
Labels['Expired'] = FeatureMortality['Expired'] 
y_values = Labels.to_numpy()
y_values = y_values.flatten()

In [ ]:
nClients = NumSubjects
X = FeatureSet

Testing RF

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from collections import Counter

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from numpy.random import seed
seed(123)

import sys
mod=sys.modules[__name__]
from sklearn.ensemble import RandomForestClassifier
model =  RandomForestClassifier(random_state=42)
from sklearn import metrics

acc_score = []
re_score = []
pre_score = []
f_score = []
auroc = []
auprc = []

from sklearn.model_selection import KFold 
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn import preprocessing

y = y_values
resample = NeighbourhoodCleaningRule()
k = 5

scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)
X_test =  scaler.transform(X) 

X = X.reshape(nClients,numberOfWindows*numberOfFeatures)
                                                   
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

In [ ]:
for train_ix, val_ix in skf.split(X,y):
    # select rows
    X_train, X_val = X[train_ix], X[val_ix]
    y_train, y_val = y[train_ix], y[val_ix]

    X_train,y_train = resample.fit_resample(X_train, y_train) 

    history4 = model.fit(X_train,y_train) 
    y_pred = model.predict(X_val)
    y_pred = list(map(lambda x: 0 if x<=0.5 else 1, y_pred))

    y_actu = pd.Series(y_val)
    y_pred = pd.Series(y_pred)
    
    f1_value = f1_score(y_actu, y_pred,pos_label = 1, average='binary') 
    sensitivity = recall_score(y_actu, y_pred,pos_label = 1, average='binary')
    precision = precision_score(y_actu, y_pred,pos_label = 1, average='binary')
    prc = average_precision_score(y_actu, y_pred,pos_label = 1)
    roc = roc_auc_score(y_actu, y_pred)
    accuracy = accuracy_score(y_actu, y_pred)
    
    acc_score.append(accuracy)
    re_score.append(sensitivity)
    pre_score.append(precision)
    f_score.append(f1_value)
    auroc.append(roc)
    auprc.append(prc)
   
avg_acc_score = sum(acc_score)/k
avg_recall_score = sum(re_score)/k
avg_precision_score = sum(pre_score)/k
avg_f1_score = sum(f_score)/k
avg_roc_score = sum(auroc)/k
avg_prc_score = sum(auprc)/k

sensitivity = avg_recall_score
precision = avg_precision_score
accuracy = avg_acc_score
f1_score = avg_f1_score
auroc = avg_roc_score
auprc = avg_prc_score


In [ ]:
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))
print('Recall of each fold - {}'.format(re_score))
print('Avg Reccall : {}'.format(avg_recall_score))
print('Precision of each fold - {}'.format(pre_score))
print('Avg Precision : {}'.format(avg_precision_score))
print('F1_Score of each fold - {}'.format(f_score))
print('Avg F1_score : {}'.format(avg_f1_score))
print('AUROC of each fold - {}'.format(auroc))
print('Avg AUROC : {}'.format(avg_roc_score))
print('AUPRC of each fold - {}'.format(auprc))
print('Avg AUPRC : {}'.format(avg_prc_score)) 

In [ ]:
import csv
import os.path
from datetime import datetime

csv_columns = ['model-type','precision','sensitivity','f1-score','accuracy','AUROC','AUPRC','NumberOfWindows']
dict_data = [{'model-type':'RF', 'precision': precision,'sensitivity': sensitivity,'f1-score': f1_score,'accuracy': accuracy,'NumberOfWindows':numberOfWindows,'AUROC':auroc,'AUPRC':auprc}]
metric_file = "Results/Results_RF.csv"

file_exists = os.path.isfile(metric_file)
try:
    with open(metric_file, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        if not file_exists:
            writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")